In [1]:
import os
import sys
import copy
import wget
import time
import asyncio
import warnings
import logging
from astropy import time as astropytime

import astropy.units as u
from astropy.coordinates import AltAz, ICRS, EarthLocation, Angle

import numpy as np

from lsst.ts import salobj
from lsst.ts.standardscripts.auxtel.attcs import ATTCS
from lsst.ts.standardscripts.auxtel.latiss import LATISS

%matplotlib inline

In [2]:
stream_handler = logging.StreamHandler(sys.stdout)

logger = logging.getLogger()
logger.addHandler(stream_handler)
logger.level = logging.DEBUG

In [3]:
attcs = ATTCS()

await attcs.start_task 

Read historical data in 4.41 sec
Read historical data in 4.88 sec
Read historical data in 5.62 sec
Read historical data in 6.46 sec
Read historical data in 7.32 sec
Read historical data in 8.21 sec
Read historical data in 9.22 sec


[None, None, None, None, None, None, None]

RemoteTelemetry(ATPtg, 0, timeAndDate) falling behind; read 12 messages


In [11]:
await attcs.enable(settings={
    'ataos': "current",
    'atmcs': "",
    'atptg': "",
    'atpneumatics': "",
    'athexapod': "auxtel_201912_crazy.yaml",
    'atdome': "test.yaml",
    'atdometrajectory': ""})

Gathering settings.
Received settings from users.: {'ataos': 'current', 'atmcs': '', 'atptg': '', 'atpneumatics': '', 'athexapod': 'auxtel_201912_crazy.yaml', 'atdome': 'test.yaml', 'atdometrajectory': ''}
Settings versions: {'ataos': 'current', 'atmcs': '', 'atptg': '', 'atpneumatics': '', 'athexapod': 'auxtel_201912_crazy.yaml', 'atdome': 'test.yaml', 'atdometrajectory': ''}
Enabling all components
Enabling  atmcs
Enabling  atptg
Enabling  ataos
Enabling  atpneumatics
Enabling  athexapod
Enabling  atdome
Enabling  atdometrajectory
[atmcs]::[<State.STANDBY: 5>, <State.DISABLED: 1>, <State.ENABLED: 2>]
[atptg]::[<State.STANDBY: 5>, <State.DISABLED: 1>, <State.ENABLED: 2>]
[ataos]::[<State.STANDBY: 5>, <State.DISABLED: 1>, <State.ENABLED: 2>]
[atpneumatics]::[<State.STANDBY: 5>, <State.DISABLED: 1>, <State.ENABLED: 2>]
[athexapod]::[<State.STANDBY: 5>, <State.DISABLED: 1>, <State.ENABLED: 2>]
[atdome]::[<State.STANDBY: 5>, <State.DISABLED: 1>, <State.ENABLED: 2>]
[atdometrajectory]::[<S

In [12]:
await attcs.prepare_for_flatfield()

Cover state <MirrorCoverState.CLOSED: 6>
Opening M1 cover.
Cover state <MirrorCoverState.STANDBY: 5>
Cover state <MirrorCoverState.DISABLED: 1>
Cover state <MirrorCoverState.CLOSED: 6>
Cover state <MirrorCoverState.INMOTION: 8>
Cover state <MirrorCoverState.OPENED: 7>
Flushing events
Sending command
Scheduling check coroutines
process as completed...
[Telescope] delta Alt = -040.977 | delta Az= -154.319
[Telescope] delta Alt = -038.530 | delta Az= -156.777
[Telescope] delta Alt = -034.532 | delta Az= -160.774
[Telescope] delta Alt = -030.531 | delta Az= -164.773
[Telescope] delta Alt = -026.538 | delta Az= -168.773
[Telescope] delta Alt = -022.584 | delta Az= -172.774
[Telescope] delta Alt = -018.734 | delta Az= -176.774
[Telescope] delta Alt = -015.056 | delta Az= +179.227
[Telescope] delta Alt = -011.638 | delta Az= +175.227
[Telescope] delta Alt = -008.568 | delta Az= +171.226
[Telescope] delta Alt = -005.920 | delta Az= +167.227
[Telescope] delta Alt = -004.775 | delta Az= +165.227

TimeoutError: 

In [ ]:
await attcs.shutdown()

In [ ]:
attcs.dome_slew_tolerance = Angle(5.*u.deg)

In [ ]:
await attcs.slew_icrs(ra="03 00 00", dec="-45 00 00", rot_pa=0., target_name="Test")

In [ ]:
await attcs.point_azel(target_name="Park position",
                       az=attcs.tel_park_az,
                       el=attcs.tel_park_el,
                       wait_dome=False)


In [ ]:
await attcs.startup(settings={
    'ataos': "current",
    'atmcs': "",
    'atptg': "",
    'atpneumatics': "",
    'athexapod': "auxtel_201912_crazy.yaml",
    'atdome': "test.yaml",
    'atdometrajectory': ""})

In [ ]:
await attcs.ataos.cmd_enableCorrection.set_start(hexapod=True)

In [4]:
latiss = LATISS(attcs.domain)

In [5]:
await latiss.start_task

Read historical data in 0.06 sec
Read historical data in 0.36 sec
Read historical data in 0.62 sec
Read historical data in 1.13 sec


[None, None, None, None]

In [ ]:
await salobj.set_summary_state(latiss.atheaderservice, salobj.State.ENABLED)

In [ ]:
await latiss.take_image(exptime=2., shutter=True, image_type="OBJECT", group_id=astropytime.Time.now().tai.isot)

In [ ]:
await latiss.take_image

In [ ]:
ack = await attcs.atdometrajectory.cmd_standby.start()

In [ ]:
print(ack.private_sndStamp)

In [ ]:
print(ack)

In [13]:
await salobj.set_summary_state(latiss.atarchiver, salobj.State.ENABLED, settingsToApply="normal")

[<State.FAULT: 3>, <State.STANDBY: 5>, <State.DISABLED: 1>, <State.ENABLED: 2>]

In [ ]:
bias = await latiss.take_bias(nbias=1)

In [ ]:
print(bias)

In [ ]:
image_1 = await latiss.take_image(exptime=2., shutter=True, image_type="OBJECT", group_id=astropytime.Time.now().tai.isot)

In [ ]:
_, _, prefix, suffix = image_1.imageName.split("_")

In [ ]:
        prefix = tmp[2]  # dayobs without the dashes

        # Don't remember why I used int here... whitespace?
        # surely fixable but bigger fish.
        suffix = '{:05d}'.format(int(tmp[3].split('-')[0]))  # SEQNUM, but need to trim extra 0 in obsid
        intra_visitID = int((prefix + suffix))


In [ ]:
prefix, suffix

In [ ]:
# suffix = '{:05d}'.format(int(tmp[3].split('-')[0]))  # SEQNUM, but need to trim extra 0 in obsid
intra_visitID = int((prefix + suffix[1:]))

In [ ]:
intra_visitID

In [ ]:
await latiss.take_bias(nbias=10)

In [14]:
await latiss.take_bias(nbias=10)

Waiting for header service LFO before continuing
Waiting for header service LFO before continuing
Waiting for header service LFO before continuing
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 10 messages
Waiting for header service LFO before continuing
Waiting for header service LFO before continuing
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 13 messages
Waiting for header service LFO before continuing
Waiting for header service LFO before continuing
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 11 messages
Waiting for header service LFO before continuing
Waiting for header service LFO before continuing
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 12 messages
Waiting for header service LFO before continuing


In [15]:
await latiss.take_darks(exptime=100., ndarks=10)

Waiting for header service LFO before continuing
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 10 messages
Waiting for header service LFO before continuing
Waiting for header service LFO before continuing
Waiting for header service LFO before continuing
Waiting for header service LFO before continuing
Waiting for header service LFO before continuing
Waiting for header service LFO before continuing
Waiting for header service LFO before continuing
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 23 messages
Waiting for header service LFO before continuing
Waiting for header service LFO before continuing
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 12 messages


In [16]:
await latiss.take_bias(nbias=10)

Waiting for header service LFO before continuing
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 20 messages
Waiting for header service LFO before continuing
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 10 messages
Waiting for header service LFO before continuing
Waiting for header service LFO before continuing
Waiting for header service LFO before continuing
Waiting for header service LFO before continuing
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 12 messages
Waiting for header service LFO before continuing
Waiting for header service LFO before continuing
Waiting for header service LFO before continuing
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 12 messages
Waiting for header service LFO before continuing


In [18]:
await latiss.take_flats(exptime=5., nflats=1,filter="KPNO_406_828nm",grating="empty_1")

Waiting for header service LFO before continuing
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 10 messages


In [ ]:
await latiss.take_flats

In [22]:
len(np.arange(0.1, 8., 0.1))

79

In [24]:
for exptime in np.arange(0.1, 8., 0.1):
    print(f"Take flats with {exptime}s.")
    await latiss.take_flats(exptime=exptime, 
                            nflats=2, 
                            filter="KPNO_406_828nm",
                            grating="empty_1")

    await latiss.take_flats(exptime=1.0, 
                           nflats=2, 
                           filter="KPNO_406_828nm",
                           grating="empty_1")


Take flats with 0.1s.
Waiting for header service LFO before continuing
Waiting for header service LFO before continuing
Waiting for header service LFO before continuing
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 10 messages
Waiting for header service LFO before continuing
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 12 messages
Take flats with 0.2s.
Waiting for header service LFO before continuing
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 16 messages
Waiting for header service LFO before continuing
Waiting for header service LFO before continuing
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 10 messages
Waiting for header service LFO before continuing
Take flats with 0.30000000000000004s.
Waiting for header service LFO before continuing
Waiting for header service LFO before continuing
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 13 messages
Waiting for header service LFO before conti

In [25]:
await latiss.take_bias(nbias=10)

Waiting for header service LFO before continuing
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 11 messages
Waiting for header service LFO before continuing
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 18 messages
Waiting for header service LFO before continuing
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 11 messages
Waiting for header service LFO before continuing
Waiting for header service LFO before continuing
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 11 messages
Waiting for header service LFO before continuing
Waiting for header service LFO before continuing
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 12 messages
Waiting for header service LFO before continuing
Waiting for header service LFO before continuing
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 11 messages
Waiting for header service LFO before continuing


In [26]:
# test
await latiss.take_flats(exptime=5, 
                        nflats=1, 
                        filter="KPNO_406_828nm",
                        grating="ronchi170lpmm")

Waiting for header service LFO before continuing


In [27]:
await latiss.take_flats(exptime=5, 
                        nflats=10, 
                        filter="KPNO_406_828nm",
                        grating="ronchi170lpmm")

Waiting for header service LFO before continuing
Waiting for header service LFO before continuing
Waiting for header service LFO before continuing
Waiting for header service LFO before continuing
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 28 messages
Waiting for header service LFO before continuing
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 10 messages
Waiting for header service LFO before continuing
Waiting for header service LFO before continuing
Waiting for header service LFO before continuing
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 12 messages
Waiting for header service LFO before continuing
Waiting for header service LFO before continuing


In [30]:
# Test
await latiss.take_flats(exptime=5, 
                        nflats=1, 
                        filter="KPNO_406_828nm",
                        grating="ronchi90lpmm")

Waiting for header service LFO before continuing
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 10 messages


In [31]:
await latiss.take_bias(nbias=10)

Waiting for header service LFO before continuing
Waiting for header service LFO before continuing
Waiting for header service LFO before continuing
Waiting for header service LFO before continuing
Waiting for header service LFO before continuing
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 13 messages
Waiting for header service LFO before continuing
Waiting for header service LFO before continuing
Waiting for header service LFO before continuing
Waiting for header service LFO before continuing
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 13 messages
Waiting for header service LFO before continuing


In [34]:
await latiss.take_flats(exptime=5, 
                        nflats=10, 
                        filter="KPNO_406_828nm",
                        grating="ronchi90lpmm")

Waiting for header service LFO before continuing
Waiting for header service LFO before continuing
Waiting for header service LFO before continuing
Waiting for header service LFO before continuing
Waiting for header service LFO before continuing
Waiting for header service LFO before continuing
Waiting for header service LFO before continuing
Waiting for header service LFO before continuing
Waiting for header service LFO before continuing
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 11 messages
Waiting for header service LFO before continuing


In [37]:
# Test
await latiss.take_flats(exptime=1.5, 
                        nflats=1, 
                        filter="blank_bk7_wg05",
                        grating="empty_1")

Waiting for header service LFO before continuing
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 13 messages


In [36]:
await latiss.take_bias(nbias=10)

Waiting for header service LFO before continuing
Waiting for header service LFO before continuing
Waiting for header service LFO before continuing
Waiting for header service LFO before continuing
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 10 messages
Waiting for header service LFO before continuing
Waiting for header service LFO before continuing
Waiting for header service LFO before continuing
Waiting for header service LFO before continuing
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 10 messages
Waiting for header service LFO before continuing
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 14 messages
Waiting for header service LFO before continuing


In [38]:
await latiss.take_flats(exptime=1.5, 
                        nflats=10, 
                        filter="blank_bk7_wg05",
                        grating="empty_1")

Waiting for header service LFO before continuing
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 13 messages
Waiting for header service LFO before continuing
Waiting for header service LFO before continuing
Waiting for header service LFO before continuing
Waiting for header service LFO before continuing
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 10 messages
Waiting for header service LFO before continuing
Waiting for header service LFO before continuing
Waiting for header service LFO before continuing
Waiting for header service LFO before continuing
Waiting for header service LFO before continuing


In [41]:
# Test 
await latiss.take_flats(exptime=5., 
                        nflats=1, 
                        filter="blank_bk7_wg05",
                        grating="ronchi170lpmm")

Waiting for header service LFO before continuing
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 10 messages


In [40]:
await latiss.take_bias(nbias=10)

Waiting for header service LFO before continuing
Waiting for header service LFO before continuing
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 11 messages
Waiting for header service LFO before continuing
Waiting for header service LFO before continuing
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 12 messages
Waiting for header service LFO before continuing
Waiting for header service LFO before continuing
Waiting for header service LFO before continuing
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 21 messages
Waiting for header service LFO before continuing
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 10 messages
Waiting for header service LFO before continuing
Waiting for header service LFO before continuing


In [42]:
await latiss.take_flats(exptime=4., 
                        nflats=10, 
                        filter="blank_bk7_wg05",
                        grating="ronchi170lpmm")

Waiting for header service LFO before continuing
Waiting for header service LFO before continuing
Waiting for header service LFO before continuing
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 13 messages
Waiting for header service LFO before continuing
Waiting for header service LFO before continuing
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 11 messages
Waiting for header service LFO before continuing
Waiting for header service LFO before continuing
Waiting for header service LFO before continuing
Waiting for header service LFO before continuing
Waiting for header service LFO before continuing
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 11 messages


In [43]:
# Test
await latiss.take_flats(exptime=5.,
                        nflats=1, 
                        filter="blank_bk7_wg05",
                        grating="ronchi90lpmm")

Waiting for header service LFO before continuing
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 11 messages


In [44]:
await latiss.take_bias(nbias=10)

Waiting for header service LFO before continuing
Waiting for header service LFO before continuing
Waiting for header service LFO before continuing
Waiting for header service LFO before continuing
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 13 messages
Waiting for header service LFO before continuing
Waiting for header service LFO before continuing
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 12 messages
Waiting for header service LFO before continuing
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 10 messages
Waiting for header service LFO before continuing
Waiting for header service LFO before continuing
Waiting for header service LFO before continuing
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 15 messages


In [45]:
await latiss.take_flats(exptime=3.5, 
                        nflats=10, 
                        filter="blank_bk7_wg05",
                        grating="ronchi90lpmm")

Waiting for header service LFO before continuing
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 10 messages
Waiting for header service LFO before continuing
Waiting for header service LFO before continuing
Waiting for header service LFO before continuing
Waiting for header service LFO before continuing
Waiting for header service LFO before continuing
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 17 messages
Waiting for header service LFO before continuing
Waiting for header service LFO before continuing
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 20 messages
Waiting for header service LFO before continuing
Waiting for header service LFO before continuing


In [46]:
await latiss.take_flats(exptime=1.5, 
                        nflats=1, 
                        filter="empty_3",
                        grating="empty_1")

Waiting for header service LFO before continuing
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 12 messages


In [47]:
await latiss.take_bias(nbias=10)

Waiting for header service LFO before continuing
Waiting for header service LFO before continuing
Waiting for header service LFO before continuing
Waiting for header service LFO before continuing
Waiting for header service LFO before continuing
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 13 messages
Waiting for header service LFO before continuing
Waiting for header service LFO before continuing
Waiting for header service LFO before continuing
Waiting for header service LFO before continuing
Waiting for header service LFO before continuing
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 12 messages


In [48]:
await latiss.take_flats(exptime=1.5, 
                        nflats=50, 
                        filter="empty_3",
                        grating="empty_1")

Waiting for header service LFO before continuing
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 11 messages
Waiting for header service LFO before continuing
Waiting for header service LFO before continuing
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 10 messages
Waiting for header service LFO before continuing
Waiting for header service LFO before continuing
Waiting for header service LFO before continuing
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 10 messages
Waiting for header service LFO before continuing
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 12 messages
Waiting for header service LFO before continuing
Waiting for header service LFO before continuing
Waiting for header service LFO before continuing
Waiting for header service LFO before continuing
Waiting for header service LFO before continuing
Waiting for header service LFO before continuing
RemoteEvent(ATHeaderService, 0, logMessage) falling beh

In [49]:
await latiss.take_bias(nbias=10)

Waiting for header service LFO before continuing
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 12 messages
Waiting for header service LFO before continuing
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 11 messages
Waiting for header service LFO before continuing
Waiting for header service LFO before continuing
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 13 messages
Waiting for header service LFO before continuing
Waiting for header service LFO before continuing
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 14 messages
Waiting for header service LFO before continuing
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 10 messages
Waiting for header service LFO before continuing
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 10 messages
Waiting for header service LFO before continuing
Waiting for header service LFO before continuing


In [ ]:
await latiss.take_darks(exptime=100., ndarks=10)

Waiting for header service LFO before continuing
Waiting for header service LFO before continuing
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 10 messages
